# Mix both readers to match the seizure time
Ashes to ashes - David Bowie

In [9]:
pip install h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 95.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd

df_diary = pd.read_csv('/home/tperezsanchez/FoundationModel_EEG_Dissertation/EEG_data_vis/data/Working/XB47Y/preprocessCSV_XB47Y/diary.csv')

# Para ver las primeras filas
print(df_diary.head())

             Timestamp          timestamp_formatted
0  2019-10-31 23:05:29  [[2019, 10, 31, 23, 5, 29]]
1  2019-11-01 10:15:24  [[2019, 11, 1, 10, 15, 24]]
2  2019-11-01 16:00:14   [[2019, 11, 1, 16, 0, 14]]
3  2019-11-08 22:50:24  [[2019, 11, 8, 22, 50, 24]]
4  2019-11-09 11:20:52  [[2019, 11, 9, 11, 20, 52]]


## Functions

In [9]:
# first version
import re
import numpy as np

def date_checker(date1, date2):
    def extract_numbers(val):
  
        val_str = str(val)
        
        numbers = re.findall(r'\d+', val_str)
        
        return [int(n) for n in numbers]

    try:
        list1 = extract_numbers(date1)
        list2 = extract_numbers(date2)
        
        return list1 == list2
    except Exception as e:
        print(f"Error comparando: {e}")
        return False

#test
formato_comas = "[[2019, 10, 31, 23,    5, 29]"
formato_espacios = "[[2019   10   31            23   5   29]]"

if date_checker(formato_comas, formato_espacios):
    print("si")
else:
    print("no")

si


In [16]:
#second version with time range 5 min
import re
from datetime import datetime, timedelta

def date_checker(date1, date2):
    def to_datetime(val):
        # Extraemos los números: [año, mes, día, hora, minuto, segundo]
        val_str = str(val)
        nums = [int(n) for n in re.findall(r'\d+', val_str)]
        
        # Creamos el objeto datetime (asumiendo el orden estándar de tus strings)
        # Tomamos los primeros 6 elementos por si el string tiene extras
        return datetime(nums[0], nums[1], nums[2], nums[3], nums[4], nums[5])

    try:
        dt1 = to_datetime(date1)
        dt2 = to_datetime(date2)
        
        # Calculamos la diferencia absoluta
        diferencia = abs(dt1 - dt2)
        
        # Verificamos si la diferencia es menor o igual a 5 minutos
        return diferencia <= timedelta(minutes=5)
        
    except Exception as e:
        print(f"Error comparando: {e}")
        return False

# --- Pruebas ---
# Caso 1: Idénticas
fecha_a = "[[2019, 10, 31, 23, 5, 29]"
fecha_b = "[[2019  10  31            23   5   29]]"

# Caso 2: Con 4 minutos de diferencia
fecha_c = "[[2019, 10, 31, 23, 0, 29]" 

print(f"¿Son iguales? {date_checker(fecha_a, fecha_b)}") # True
print(f"¿Están en el rango de 5 min? {date_checker(fecha_a, fecha_c)}") # True

¿Son iguales? True
¿Están en el rango de 5 min? True


In [14]:
import os
import numpy as np
from datetime import datetime
from scipy.io import loadmat

def save_eeg_table_with_T0(mat_path, EEG_Table, output_dir="."):
    """
    Save an EEG table to CSV using the .mat filename and the T0 timestamp.

    Parameters
    ----------
    mat_path : str
        Full path to the .mat file.
    EEG_Table : pandas.DataFrame
        DataFrame containing Time + EEG channels.
    output_dir : str, optional
        Directory where the CSV will be saved (default: current directory).

    Returns
    -------
    csv_path : str
        Full path to the saved CSV file.
    """

    # Load .mat to access header
    mat_data = loadmat(mat_path)
    hdr = mat_data["hdr"]

    # Extract T0
    orig = hdr["orig"][0,0]
    T0 = np.array(orig["T0"][0,0]).astype(int).flatten()

    start_time = datetime(
        year=T0[0],
        month=T0[1],
        day=T0[2],
        hour=T0[3],
        minute=T0[4],
        second=T0[5]
    )

    # Format timestamp
    date_str = start_time.strftime("%Y%m%d_%H%M%S")

    # Base filename (without .mat)
    base_name = os.path.splitext(os.path.basename(mat_path))[0]

    # Final CSV name
    csv_name = f"{base_name}_{date_str}.csv"
    csv_path = os.path.join(output_dir, csv_name)

    # Save
    EEG_Table.to_csv(csv_path, index=False)

    return csv_path
# how to use it:
#csv_path = save_eeg_table_with_T0(
#    mat_path="/home/tperezsanchez/data/XB47Y_1.mat",
#    EEG_Table=EEG_Table,
#    output_dir="/home/tperezsanchez/data/csv"
#)

In [17]:
import os
import h5py
from scipy.io import loadmat

folder_path = "/home/tperezsanchez/FoundationModel_EEG_Dissertation/EEG_data_vis/data/Working/XB47Y/"

for filename in os.listdir(folder_path):
    if not filename.lower().endswith(".mat") or filename.startswith('.'):
        continue
        
    file_path = os.path.join(folder_path, filename)
    
    try:
        # 1. Try standard Scipy loading (v4, v6, v7)
        data_XB47Y = loadmat(file_path)
        hdr = data_XB47Y["hdr"]
        # Scipy loads structs as numpy object arrays
        orig = hdr["orig"][0, 0]
        T0 = orig["T0"][0, 0]
        for seizure_date in df_diary["timestamp_formatted"]:
            #print(seizure_date)
            #print(T0)
            if date_checker(seizure_date,T0):
                print('hola')
                fs = float(hdr['Fs'][0,0]) # transform to float32, this is the sampling freq/rate
                channels_raw = hdr['label'][0,0] # chanells, in this case they are two
                channels_raw.shape # 2 rows and 1 columns 
                channels_raw.shape[0] # I select only the channels
                channels = []
                for row in channels_raw:
                    channels.append(str(row[0][0]))
                    # row[0][0] --> EEG SQ_D-SQ_C
                signal = np.asarray(data_XB47Y['data'])
                if signal.shape[1] != len(channels) and signal.shape[0] == len(channels):
                    signal = signal.T
                # Converst signal from .mat into a numpy array
                
                
                #number of samples
                n_samples = signal.shape[0] 
                print("number of samples (rows):",n_samples)
                
                # Δt = 1 / freq ≈ 0.00483 s
                time = np.arange(n_samples) / fs 
                
                # create data frame:
                EEG_Table = pd.DataFrame(signal, columns=channels)
                EEG_Table.insert(0, "Time", time)
                
                csv_path = save_eeg_table_with_T0(
                mat_path=file_path,
                EEG_Table=EEG_Table,
                output_dir="/home/tperezsanchez/EEG_data_vis/results/Seiz_test_04022026/"
            )
    except (ValueError, NotImplementedError):
        # 2. Try HDF5 loading (v7.3)
        # Note: h5py handles the data structure differently (often transposed)
        with h5py.File(file_path, 'r') as f:
            # Accessing HDF5 datasets is like a dictionary
            # Depending on how the struct was saved, you may need to adjust indices
            hdr = f['hdr']
            orig = hdr['orig']
            # HDF5 often stores references; this is a simplified access:
            T0 = f[orig['T0'][0, 0]][()] 
            for seizure_date in df_diary["timestamp_formatted"]:
                print(seizure_date)
            
    #print(f"File: {filename} | T0: {T0}")



### VERSION 2

In [18]:
#VERSION 2
import os
import h5py
import numpy as np
import pandas as pd
from scipy.io import loadmat

folder_path = "/home/tperezsanchez/FoundationModel_EEG_Dissertation/EEG_data_vis/data/Working/XB47Y/"

for filename in os.listdir(folder_path):
    if not filename.lower().endswith(".mat") or filename.startswith('.'):
        continue
        
    file_path = os.path.join(folder_path, filename)
    data_loaded = False
    
    # --- PASO 1: CARGAR EL ARCHIVO (Solamente carga) ---
    try:
        data_mat = loadmat(file_path)
        hdr = data_mat["hdr"]
        # Extraer T0 para Scipy
        T0 = hdr["orig"][0, 0]["T0"][0, 0]
        fs = float(hdr['Fs'][0,0].item())
        signal = np.asarray(data_mat['data'])
        channels_raw = hdr['label'][0,0]
        is_hdf5 = False
        data_loaded = True
    except (ValueError, NotImplementedError, KeyError):
        try:
            # Intento con HDF5 si Scipy falla
            with h5py.File(file_path, 'r') as f:
                # Nota: Acceder a HDF5 requiere dereferenciar objetos
                # Esto es un ejemplo, v7.3 suele ser más complejo:
                T0_ref = f['hdr']['orig']['T0'][0, 0]
                T0 = f[T0_ref][()] 
                # ... (habría que cargar el resto de variables igual)
                is_hdf5 = True
                data_loaded = True
        except Exception as e:
            print(f"No se pudo cargar {filename}: {e}")
            continue

    # --- PASO 2: PROCESAR (Fuera del Try de carga) ---
    if data_loaded:
        for seizure_date in df_diary["timestamp_formatted"]:
            # Usando la función date_checker con Regex que hicimos antes
            if date_checker(T0, seizure_date):
                print(f"¡Match! Procesando {filename} para la fecha {seizure_date}")
                
                # Extraer canales
                channels = [str(row[0][0]) for row in channels_raw]
                
                # Corregir transposición de señal si es necesario
                if signal.shape[1] != len(channels) and signal.shape[0] == len(channels):
                    signal = signal.T
                
                n_samples = signal.shape[0]
                time = np.arange(n_samples) / fs 
                
                # Crear DataFrame
                EEG_Table = pd.DataFrame(signal, columns=channels)
                EEG_Table.insert(0, "Time", time)
                
                # Guardar
                save_eeg_table_with_T0(
                    mat_path=file_path,
                    EEG_Table=EEG_Table,
                    output_dir="/home/tperezsanchez/EEG_data_vis/results/Seiz_test_04022026/"
                )

In [15]:
for filename in os.listdir(folder_path):
    if not filename.lower().endswith(".mat") or filename.startswith('.'):
        continue
        
    file_path = os.path.join(folder_path, filename)
    print(f"\n🔍 Analizando: {filename}")
    
    data_loaded = False
    
    try:
        data_mat = loadmat(file_path)
        hdr = data_mat["hdr"]
        T0 = hdr["orig"][0, 0]["T0"][0, 0]
        # Usamos item() para evitar el warning
        fs = float(hdr['Fs'][0,0].item())
        signal = np.asarray(data_mat['data'])
        channels_raw = hdr['label'][0,0]
        data_loaded = True
        print(f"  ✅ Cargado con Scipy. T0 encontrado: {T0}")
    except Exception as e:
        print(f"  ❌ Falló Scipy: {type(e).__name__} - {e}")
        # Aquí podrías poner el bloque de h5py si sabes que tienes archivos v7.3
        continue

    if data_loaded:
        match_encontrado = False
        for seizure_date in df_diary["timestamp_formatted"]:
            if date_checker(seizure_date, T0):
                match_encontrado = True
                print(f"  🎯 ¡MATCH! Fecha diario: {seizure_date} == T0: {T0}")
                
                try:
                    # Debug de canales
                    print(f"  📏 Dimensiones señal: {signal.shape}")
                    channels = [str(row[0][0]) for row in channels_raw]
                    print(f"  🏷️ Canales extraídos: {channels}")

                    if signal.shape[1] != len(channels) and signal.shape[0] == len(channels):
                        signal = signal.T
                    
                    n_samples = signal.shape[0]
                    time = np.arange(n_samples) / fs 
                    
                    EEG_Table = pd.DataFrame(signal, columns=channels)
                    EEG_Table.insert(0, "Time", time)
                    
                    print(f"  💾 Intentando guardar CSV...")
                    save_eeg_table_with_T0(
                        mat_path=file_path,
                        EEG_Table=EEG_Table,
                        output_dir="/home/tperezsanchez/EEG_data_vis/results/Seiz_test_04022026/"
                    )
                    print(f"  🎉 Guardado con éxito.")
                except Exception as e:
                    print(f"  💥 Error procesando datos internos: {e}")
        
        if not match_encontrado:
            print(f"  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.")




🔍 Analizando: XB47Y_38.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   10   30    1   54   13]]
  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.

🔍 Analizando: XB47Y_34.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   12   13   10   49   58]]
  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.

🔍 Analizando: XB47Y_133.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   11   17   16   24    8]]
  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.

🔍 Analizando: XB47Y_110.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   11    6    7   42   43]]
  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.

🔍 Analizando: XB47Y_15.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   12    2   21   14   31]]
  ⚠️ No se encontró ninguna coincidencia en el diario para este archivo.

🔍 Analizando: XB47Y_129.mat
  ✅ Cargado con Scipy. T0 encontrado: [[2019   11   16   11   25   56]]
  ⚠️ No se encontró ninguna coin

In [ ]:
hdr = data_XB47Y_1['hdr']
orig = hdr['orig'][0,0]
T0 = orig['T0'][0,0]
